# <h1 align="center"> THEME 5 - Résolution numérique de systèmes dynamiques </h1>

### 🎯 Objectifs

- Résoudre des ODE linéaires et non-linéaires
- Résoudre un système d'ODE

### 📚 Notions 

**Essentielles:**

- [Exemple 1](#ex1): Réaction non-linéaire
    - Introduire à la fonction solve_ivp().
    - Construire la fonction de l'ODE.
    - Résoudre une ODE numériquement.
- [Exemple 2](#ex2): Cinématique Michaelis-Menten
    - Construire une fonction pour un système d'ODE.
    - Résoudre un système d'ODE numériquement.

**Avancées:**

- [Exemple 3](#ex3): Système masse-ressort amorti
    - Modifier la précision de la résolution numérique.
    - Notions de tolérance absolue et relative.
- [Exemple 4](#ex4): ODE raide
    - Observer la stabilité dans la résolution numérique
    - Méthodes implicites et explicites.
    
Un [lexique](#lexique) avec l'ensemble des fonctions qui ont été vues est disponible à la fin du notebook.

### 🧰 Librairies

- **Scipy**: est une librairie Python open-source utilisée pour le calcul scientifique. SciPy contient des modules pour l'optimisation, l'algèbre linéaire, l'intégration, l'interpolation, les fonctions spéciales, la FFT, le traitement du signal et de l'image, les solveurs ODE et d'autres tâches courantes en sciences et en ingénierie.

### 🔗 Référence

- [Documentation Scipy](https://docs.scipy.org/doc/scipy/reference/index.html#scipy-api)

### ⚙️ Installation

`pip install scipy`

## <a name="ex1"><h2 align="center"> Exemple 1 - Réaction Non-Linéaire </h2></a>

### 📝 Contexte

Soit la réaction suivante:

$$
2 A \rightarrow B
$$

Son équation est une ODE non-linéaire:

$$
\frac{d C_{A}}{d t}=-k C_{A}^{2}
$$

### 🧪 Paramètres
 
Donnée:
- $k=2 \, L.mol^{-1}.min^{-1}$

Condition initiale:
- $C_{A0}=1 \, mol.L^{-1}$

### ⭐ Objectif

Résoudre l'ODE numériquement et tracer la concentration de A entre t=0 et t=10 min avec 26 points.

### 💻 Code

La résolution numérique d'ODEs à conditions initiales avec Scipy est très simple puisque qu'elle necessite qu'une seule fonction: [`solve_ivp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html#scipy-integrate-solve-ivp). Cette fonction est disponible dans le sous-module `scipy.integrate`.

In [ ]:
import numpy as np
import scipy.integrate as spi
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 5)  # Augmenter la taille des figures

L'utilisation de `solve_ivp` necessite 3 arguments obligatoires:

- `fun`: Une fonction qui décrit le système d'ODE de la forme suivante: $\frac{d \vec{y}}{dt} = f(t, \vec{y})$
- `t_span`: Un intervalle de temps d'intégration fourni avec un tuple: `(t0, tf)`.
- `y0`: Le vecteur des conditions initiales $\vec{y_{0}}$.

La résolution d'un système d'ODE est vu dans l'exemple 2, cela utilise les mêmes arguments sauf que la fonction prend un vecteur y comme argument et les conditions initiales sont un vecteur.

Notez ici que le pas de temps est déterminé automatiquement mais peut être spécifié en passant à `solve_ivp` un vecteur qui contient les valeurs de temps ou l'on veut évaluer l'ODE avec l'argument (falcultatif) `t_eval`.

In [ ]:
# Définition de la fonction
# --------------------------------------------------
# Le 1er argument doit toujours être t
# Le 2ème argument doit toujours être la variable intégrée
# Le reste des arguments sont considérés comme supplémentaires
# Doit toujours retourner la valeur évaluée (à gauche de l'équation)
def f(t, C, k):
    """
    ODE de la réaction 2A -> B
    """
    dC = -k * C**2
    return dC


# Définition des paramètres
# 🕹️ --------------------------------------------------
t0 = 0  # Temps initial
tf = 10  # Temps final
k = 2  # Débit de la réaction
C0 = 1  # Concentration initiale
vec_t = np.linspace(t0, tf, 26)  # Discrétisation du temps

# Résolution de l'ODE
# --------------------------------------------------
# Rappel: si un tuple ne contient qu'une seule valeur, il faut ajouter une virgule: (x,)
sim1 = spi.solve_ivp(
    f,  # Fonction de l'ODE
    t_span=(t0, tf),  # Intervalle de temps
    y0=(C0,),  # Condition initiale
    args=(k,),  # Arguments supplémentaires de la fonction
    t_eval=vec_t,  # (optionnel) Vecteur de temps où l'ODE doit être évaluée
)

Le résultat de la résolution est un objet qui contient plusieurs données comme:

- `nfev`: Le nombre de fois que la fonction a été évaluée.
- `njev`: Le nombre de fois que le Jacobien a été évalué.
- `success`: `True` si le solveur a réussi, `False` sinon.
- `t`: Le vecteur des valeurs de temps où l'ODE a été évaluée.
- `y`: La matrice des valeurs de la solution de l'ODE.

In [ ]:
print(sim1)

Il faut noter que `sim1.y` est une matrice qui contient la solution d'une variable sur chaque ligne. Puisque dans cet exemple on résout une seule équation, alors la matrice est de dimension 1xN et pour accéder à notre solution il faut indexer la première ligne de la matrice.

Enfin, on trace la solution avec Matplotlib.

In [ ]:
# Tracer la courbe de concentration de A
plt.plot(sim1.t, sim1.y[0])
plt.xlabel("t (min)")
plt.ylabel(r"$C_{A}$ (mol/L)")
plt.title("Évolution de la concentration de A")
plt.show()

## <a name="ex2"><h2 align="center"> Exemple 2 - Cinématique Michaelis-Menten </h2></a>

### 📝 Contexte

En 1913, les chercheurs Leonor Michaelis et Maud Menten ont proposé un modèle mathématique pour décrire la vitesse d'une réaction enzymatique basée sur sa cinématique. Cette réaction implique qu'une enzyme, E, se lie à un substrat, S, pour former un complexe, ES, qui libère à son tour un produit, P, régénérant l'enzyme d'origine.

$$
\begin{aligned}
\mathrm{E}+\mathrm{S} \underset{k_{r}}{\stackrel{k_{f}}{\rightleftharpoons}} \mathrm{ES} \stackrel{k_{\mathrm{cat}}}{\longrightarrow} \mathrm{E}+\mathrm{P}
\end{aligned}
$$

Où $k_{f}$ (constante de vitesse en avant), $k_{r}$ (constante de vitesse en arrière) et $k_{cat}$ (constante de vitesse catalytique) désignent les constantes de vitesse, les doubles flèches entre S (substrat) et ES (complexe enzyme-substrat) représentent le fait que la liaison enzyme-substrat est un processus réversible, et la flèche unique en avant représente la formation de P (produit).

En appliquant la loi de conservation de la masse on obtient 4 ODEs non-linéaires:

$$
\begin{aligned}
\frac{\mathrm{d}[\mathrm{E}]}{\mathrm{d} t} &=-k_{f}[\mathrm{E}][\mathrm{S}]+k_{r}[\mathrm{ES}]+k_{\mathrm{cat}}[\mathrm{ES}] \\
\frac{\mathrm{d}[\mathrm{S}]}{\mathrm{d} t} &=-k_{f}[\mathrm{E}][\mathrm{S}]+k_{r}[\mathrm{ES}] \\
\frac{\mathrm{d}[\mathrm{ES}]}{\mathrm{d} t} &=k_{f}[\mathrm{E}][\mathrm{S}]-k_{r}[\mathrm{ES}]-k_{\mathrm{cat}}[\mathrm{ES}] \\
\frac{\mathrm{d}[\mathrm{P}]}{\mathrm{d} t} &=k_{\mathrm{cat}}[\mathrm{ES}]
\end{aligned}
$$

### 🧪 Paramètres

Données: 
- $k_{f} = 10^{-3} \, mol.s^{-1}$
- $k_{r} = 10^{-4} \, mol.s^{-1}$
- $k_{cat} = 0.1 \, mol.s^{-1}$

Conditions initiales: 
- $E_{0} = 200$ 
- $S_{0} = 500$
- $ES_{0} = 0$
- $P_{0} = 0$

### ⭐ Objectif



Résoudre le système d'ODEs numériquement et tracer la concentration de chaque espèce entre t=0 et t=50s. 

### 💻 Code

La résolution du système est assez similaire à l'exemple 1 à quelques exceptions pour accommoder le fait que nous sommes en train de résoudre un système d'ODEs cette fois.

- Le deuxième argument de notre fonction doit être un vecteur $\vec{y}$ qui contient chaque variable du système.
- La fonction doit retourner un vecteur qui contient les dérivées de premier ordre: $\frac{d \vec{y}}{dt}$

In [ ]:
# Définition de la fonction
# --------------------------------------------------
def reaction_enzymatique(t, Y, k_f, k_r, k_cat):
    """
    Système de la réaction enzymatique
    Y: vecteur des concentrations -> [E,S,ES,P]
    """
    dy = np.zeros(4)  # Création d'un vecteur de zéros [E,S,ES,P]
    e, s, es, p = Y  # Séparation des variables (équivalent à Y[0], Y[1], Y[2], Y[3])

    dy[0] = -k_f * e * s + (k_r + k_cat) * es  # dE/dt
    dy[1] = -k_f * e * s + k_r * es  # dS/dt
    dy[2] = k_f * e * s - (k_r + k_cat) * es  # dES/dt
    dy[3] = k_cat * es  # dP/dt
    return dy


# Définition des paramètres
# --------------------------------------------------
k_f = 1e-3
k_r = 1e-4
k_cat = 0.1
k = (k_f, k_r, k_cat)  # Tuple des paramètres k
t = np.linspace(0, 50, 100)
E0 = 200
S0 = 500
y0 = (E0, S0, 0, 0)  # Tuple [E,S,ES,P]

# Résolution de l'ODE
# --------------------------------------------------
sim2 = spi.solve_ivp(reaction_enzymatique, t_span=(0, 50), y0=y0, args=k)

In [ ]:
# Tracer les courbes de concentration
plt.plot(sim2.t, sim2.y[0], label="E")
plt.plot(sim2.t, sim2.y[1], label="S")
plt.plot(sim2.t, sim2.y[2], label="ES")
plt.plot(sim2.t, sim2.y[3], label="P")
plt.legend()
plt.title("Réaction enzymatique")
plt.xlabel("t (s)")
plt.ylabel("Concentration (mol/L)")
plt.grid(True)
plt.show()

## <a name="ex3"><h2 align="center"> Exemple 3 - Système masse-ressort amorti</h2></a>

### 📝 Contexte
<center>
<img src="assets/mass_spring_damper.png" height=300px/>
</center>

Dans un système masse-ressort amorti, une masse est connectée à un ressort et à un amortisseur qui sont eux fixés à une paroi fixe. La masse est initialement étirée de sa position naturelle et est ensuite relâc   hée. En considérant que le frottement est négligeable et en appliquant la deuxième loi de Newton sur le système, on en déduit l’équation différentielle du mouvement de la masse:

$$
\frac{d^{2} x}{d t^{2}}+ \left(\frac{c}{m}\right) \frac{d x}{d t} + \left(\frac{k}{m}\right) x=0
$$

Où $m$ est la masse, $c$ la constante de l'amortisseur et $k$ la constante de rappel du ressort.

Cette ODE de deuxième ordre peut être convertie en un système de deux équations linéaires: 

$$
\begin{aligned}
&\frac{d x}{d t} =v \\
&\frac{d v}{d t} = -\left(\frac{c}{m}\right) v -\left(\frac{k}{m}\right) x
\end{aligned}
$$

### 🧪 Paramètres
Données: 
- $m = 50g$
- $k = 2 \, N/m$
- $c = 0.1 \, kg/s$

Conditions initiales:
- $x_{0} = 2$
- $v_{0} = 0$

### ⭐ Objectif

Résoudre le système d'ODE entre t=0 et t=2s avec plusieurs valeurs de précision numérique. 

### 💻 Code

La résolution du système est analogue à la méthode utilisée à l'exemple 2. 

L'objectif de cet exemple est d'explorer comment modifier la précision de résolution d'un système d'ODE. Sans rentrer dans des détails techniques, les méthodes numériques utilisées dans les solveurs permettent de trouver une une estimation d'erreur, qui minimisée pour être inférieure au maximum de l'un des deux types de tolérances:

- `rtol`: tolérance de l'erreur relative. Cette dernière est calculée relativement à la grandeur de la valeur même. Simplement dit, cette tolérance permet de controler le nombre de chiffres significatifs des valeurs qui *ne sont pas proches* de 0. 

- `atol`: tolérance de l'erreur absolue. Cette dernière est une différence absolue entre la valeur calculée et la valeur réelle. Cette tolérance permet principalement de controler le nombre de chiffres significatifs des valeurs qui *sont proches* de 0.

Pour plus d'informations voir cette [explication](https://www.mathworks.com/help/simbio/ug/selecting-absolute-tolerance-and-relative-tolerance-for-simulation.html).

Dans Scipy, ces tolérances sont par défaut: 1e-3 pour `rtol` et 1e-6 pour `atol`. Elles peuvent être modifiées en fournissant un nouveau nombre décimal ou un vecteur de nombres décimaux pour controler la précision dans chaque équation. 

In [ ]:
# Définition de la fonction du système d'ODE
# --------------------------------------------------
def masse_ressort(t, Y, k, c, m):
    """
    Système masse-ressort simple
    """
    dy = np.zeros(2)  # [x,v]
    x, v = Y
    dy[0] = v
    dy[1] = -(k / m) * x - (c / m) * v
    return dy


# Définition des paramètres
# 🕹️ --------------------------------------------------
m = 50e-3
k = 2
c = 0.1
constantes = (k, c, m)
t0 = 0
tf = 10
t = np.linspace(t0, tf, 1000)
y0 = (5, 0)  # Tuple (x0, v0)

# Résolution de l'ODE avec différents paramètres de précision
# --------------------------------------------------
# default: rtol=1e-3, atol=1e-6 (default scipy)
# high_rtol: rtol=1e-1, atol=1e-6
# high_atol: rtol=1e-3, atol=1e-2
sim3_default = spi.solve_ivp(masse_ressort, t_span=(t0, tf), y0=y0, args=constantes, t_eval=t)
sim3_high_rtol = spi.solve_ivp(masse_ressort, t_span=(t0, tf), y0=y0, args=constantes, t_eval=t, rtol=1e-1)
sim3_high_atol = spi.solve_ivp(masse_ressort, t_span=(t0, tf), y0=y0, args=constantes, t_eval=t, atol=1e-2)

In [ ]:
# Fonction pour tracer nos courbes facilement
# --------------------------------------------------
def plot_system_x(plot_low=False, xlim=None, ylim=None):
    """
    Tracer la ou les courbes du système masse-ressort-amortisseur
    Args:
        plot_low (bool): Afficher les courbes avec un rtol/atol faible
        xlim (tuple): (xmin, xmax)
        ylim (tuple): (ymin, ymax)
    """
    if plot_low:
        plt.plot(sim3_high_rtol.t, sim3_high_rtol.y[0], label="high rtol")
        plt.plot(sim3_high_atol.t, sim3_high_atol.y[0], label="high atol")
    plt.plot(sim3_default.t, sim3_default.y[0], "--", label="default")
    plt.legend()
    if xlim:
        plt.xlim(xlim)
    if ylim:
        plt.ylim(ylim)
    plt.grid(True)
    plt.xlabel("t (s)")
    plt.ylabel("x (m)")
    plt.title("Déplacement du système masse-ressort-amortisseur x")
    plt.show()


plot_system_x()

Réduire la précision de la résolution de l'ODE est parfois idéal car cela permet de réduire le nombre de calculs numériques ce qui augmente la rapidité de résolution.

In [ ]:
# Afficher le nombre de fois que la fonction a été évaluée
# --------------------------------------------------
# Un nombre plus petit signifie un calcul plus rapide
print("Defaut: ", sim3_default.nfev)
print("High rtol: ", sim3_high_rtol.nfev)
print("High atol: ", sim3_high_atol.nfev)

On peut voir l'effet de la modification des valeurs de `rtol` et `atol` en comparant visuellement les courbes entre 2 intervalles de temps. On considère ici que la courbe `default` est la courbe de référence.

In [ ]:
# Intervalle où l'amplitude est élevée
# 🕹️ --------------------------------------------------
plot_system_x(True, (2, 4), (-1, 1))

En prenant l'intervalle de temps `[2,4]`, on peut voir que la courbe obtenue avec un `rtol` élevé est significativement moins précise. En revanche, la courbe avec un `atol` élevé est confondue avec la courbe `default` ce qui est attendu puisque que les valeurs de la courbe ne sont pas proche de zero ce qui signifie ce que ici c'est `rtol` qui contrôle l'erreur.

In [ ]:
# Intervalle où l'amplitude est faible
# 🕹️ --------------------------------------------------
plot_system_x(True, (8, 10), (0.02, -0.02))

Dans l'intervalle `[8,10]`, la masse s'est presque immobilisée et les oscillations sont très faibles et proches de 0. On voit ici que la courbe obtenue avec un `atol` élevé est cette fois très imprécise tandis que la courbe avec un `rtol` élevé est meilleure car puisque les valeurs sont proche de zero, l'algorithme de convergence utilise le paramètre `atol` pour minimiser l'erreur. 

## <a name="ex4"><h2 align="center"> Exemple 4 - ODE Raide </h2></a>

### 📝 Contexte

Un exemple classique de système d'ODE raide est l'analyse cinétique de la réaction chimique autocatalytique de Robertson qui implique 3 espèces:

$$
\begin{aligned}
&\dot{x} \equiv \frac{\mathrm{d} x}{\mathrm{~d} t}=-0.04 x+10^{4} y z \\
&\dot{y} \equiv \frac{\mathrm{d} y}{\mathrm{~d} t}=0.04 x-10^{4} y z-3 \times 10^{7} y^{2} \\
&\dot{z} \equiv \frac{\mathrm{d} z}{\mathrm{~d} t}=3 \times 10^{7} y^{2}
\end{aligned}
$$

Ce système d'ODE est de type raide puisque les constantes cinétiques ont des ordres de grandeur très différents ($10^{4}$ et $10^{7}$). 

### 🧪 Paramètres

Conditions initiales:
- $x_{0} = 1$
- $y_{0} = 0$
- $z_{0} = 0$

### ⭐ Objectif

Résoudre le système d'ODE entre t=0 et t=500s. 

### 💻 Code

La résolution d'un tel système a déjà été vu dans les exemples précédents. Ce qui est nouveau, c'est que cette fois nous avons un système d'ODEs raide ce qui peut engendrer des probèmes lors de sa résolution numérique. La fonction `solve_ivp` de Scipy prend l'argument facultatif `method` qui permet de specifier la méthode d'intégration. Il existe 2 grandes familles de méthodes numériques: 

- Méthode **Explicite**: stabilité sensible au pas de temps, résolution rapide, méthode généralement utilisée.
- Méthode **Implicite**: toujours stable, calculs plus lourds donc résolution plus lente. 

Les différences dans leur implémentation est vu dans le cours de GCH-2545.

Généralement, lorsque l'on a un système d'ODE raide, les méthodes explicites ne sont pas capables de converger vers la solution et la résolution échoue. En contrepartie, les méthodes implicites sont très efficaces face à ce genre de ODE.

Avec Scipy, on peut choisir entre plusieurs méthodes d'intégration. En général, il est recommandé d'utiliser la méthode explicite `RK45` (méthode par défaut) et la méthode implicite `Radau`.  

In [ ]:
# Définition de la fonction du système d'ODE
# --------------------------------------------------
def robertson(t, y):
    x, y, z = y
    xdot = -0.04 * x + 1.0e4 * y * z
    ydot = 0.04 * x - 1.0e4 * y * z - 3.0e7 * y**2
    zdot = 3.0e7 * y**2
    return xdot, ydot, zdot


# Définition des paramètres
# -------------------------------------------------
t0, tf = 0, 300
y0 = (1, 0, 0)

# Résolution
# --------------------------------------------------
sim4_explicite = spi.solve_ivp(robertson, (t0, tf), y0=y0, method="RK45")
sim4_implicite = spi.solve_ivp(robertson, (t0, tf), y0=y0, method="Radau")

Voyons si les deux méthodes ont convergé.

In [ ]:
print("Succès méthode explicite: ", sim4_explicite.success)
print("Succès méthode implicite: ", sim4_implicite.success)

In [ ]:
# Tracer les courbes de concentration
plt.plot(sim4_implicite.t, sim4_implicite.y[0], label="[X]")
plt.plot(sim4_implicite.t, 10**4 * sim4_implicite.y[1], label=r"$10^4\times$[Y]")
plt.plot(sim4_implicite.t, sim4_implicite.y[2], label="[Z]")
plt.xlabel("Temps (s)")
plt.ylabel("Concentration (mol/L)")
plt.title("Réaction chimique autocatalytique de Robertson")
plt.legend()
plt.grid(True)
plt.show()

## <a name="lexique"><h2 align="center"> Lexique </h2></a>

### ✔️ Vu dans l'exemple 1,2,3,4

- `spi.solve_ivp`: fonction qui permet de résoudre un système d'ODE.
